# Wedding Seating Optimization

This work was conducted by Group G:

- Daniel Caridade (20211588)
- Gonçalo Teles (20211684)
- Gonçalo Peres (20211625)
- Guilherme Godinho (20211552)

# 1. Library Importation

__`Step 1`__ Import the required libraries.

In [1]:
import pandas as pd
import random
import numpy as np
import copy
import math

# 2. Data Integration

__`Step 2`__ Importing the dataset into the file.

In [2]:
from utils.parser import load_relationship_matrix

# Load the data
relationships = load_relationship_matrix()
relationships.head()

,1,2,3,4,5,6,7,8,9,10,...,55,56,57,58,59,60,61,62,63,64
idx,,,,,,,,,,,,,,,,,,,,,
1,0,5000,0,0,700,700,0,0,0,0,...,100,100,0,0,100,100,100,0,0,0
2,5000,0,700,700,0,0,300,300,500,500,...,100,100,0,100,0,0,0,0,0,0
3,0,700,0,2000,0,0,0,0,300,300,...,0,0,0,0,0,0,0,0,0,0
4,0,700,2000,0,0,0,900,400,300,300,...,0,0,0,0,0,0,0,0,0,0
5,700,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 3. Helper functions

__`Step 3`__

In [3]:
from utils.WeddingSeatingHelper import WeddingSeatingHelper # adjust import path accordingly

# Suppose relationships is loaded before, e.g. from your parser:
helper = WeddingSeatingHelper(relationships)

## Hill Climbing testing

In [ ]:
from optimizers.HillClimbing import HillClimbingOptimizer

hc = HillClimbingOptimizer(helper)

best_fit, best_sol = hc.run(verbose=True)
print("Best fitness:", best_fit)

Current fitness: 19900
Current fitness: 24400
Current fitness: 28700
Current fitness: 32900
Current fitness: 37000
Current fitness: 40600
Current fitness: 44000
Current fitness: 47300
Current fitness: 50300
Current fitness: 53100
Current fitness: 55800
Current fitness: 58400
Current fitness: 60800
Current fitness: 62700
Current fitness: 64300
Current fitness: 65700
Current fitness: 67700
Current fitness: 68400
Current fitness: 69000
Current fitness: 69600
Current fitness: 69800
Current fitness: 69900
Best fitness: 69900


Testing for several iterations

In [ ]:
results_hc = []

for i in range(30):
    hc = HillClimbingOptimizer(helper, seed=i)  # different seed for each run
    best_fit, best_sol = hc.run()
    results_hc.append(best_fit)
    print(f"Run {i+1}: Best fitness = {best_fit}")

print(f"\nAverage fitness: {sum(results_hc)/len(results_hc):.2f}")
print(f"Best fitness: {max(results_hc)}")
print(f"Worst fitness: {min(results_hc)}")

## Simulated Annealing in classes 

In [7]:
from optimizers.SimulatedAnnealing import SimulatedAnnealingOptimizer

# Run Simulated Annealing
sa = SimulatedAnnealingOptimizer(helper)
best_fit, best_sol = sa.run(verbose=True)
print("Best fitness:", best_fit)

Current fitness: 16300
Current fitness: 21300
Current fitness: 26300
Current fitness: 30700
Current fitness: 35100
Current fitness: 39200
Current fitness: 42500
Current fitness: 46000
Current fitness: 49000
Current fitness: 51800
Current fitness: 54700
Current fitness: 57400
Current fitness: 59800
Current fitness: 62400
Current fitness: 64500
Current fitness: 66600
Current fitness: 68600
Current fitness: 70600
Current fitness: 71700
Current fitness: 74200
Current fitness: 75700
Current fitness: 76000
Current fitness: 76100
Current fitness: 76400
Current fitness: 76500
Current fitness: 76500
Current fitness: 76500
Current fitness: 76500
Current fitness: 76500
Best fitness: 76500


Testing for several iterations

In [ ]:
results_sa = []

for i in range(30):
    sa = SimulatedAnnealingOptimizer(helper, seed=i) # different seed for each run
    best_fit, best_sol = sa.run()
    results_sa.append(best_fit)
    print(f"Run {i+1}: Best fitness = {best_fit}")

# Summarize results
print(f"\nAverage fitness over 30 runs: {sum(results_sa)/len(results_sa):.2f}")
print(f"Best fitness found: {max(results_sa)}")
print(f"Worst fitness found: {min(results_sa)}")

Hyperparameter Tuning

In [ ]:
import itertools
import numpy as np

# Define ranges of parameters to test
L_options = [100, 200, 500]
k_options = [1.05, 1.1, 1.2]
c_options = [1_000_000, 5_000_000]
stop_options = [3, 5, 10]

# Number of runs per config to average out randomness
runs_per_config = 5

# Store results for each config
tuning_results = []

experiment_id = 1

for L, k, c, stop in itertools.product(L_options, k_options, c_options, stop_options):
    fitnesses = []
    print(f"\n=== Experiment {experiment_id} ===")
    print(f"L={L}, k={k}, c={c}, stop={stop}")

    for run in range(runs_per_config):
        optimizer = SimulatedAnnealingOptimizer(
            helper=helper,  # replace with your WeddingSeatingHelper instance
            L=L,
            k=k,
            c=c,
            stop=stop,
            seed=42 + run  # different seed per run for diversity
        )
        best_fitness, best_solution = optimizer.run(verbose=False)
        fitnesses.append(best_fitness)
        print(f" Run {run+1}: fitness = {best_fitness}")

    avg_fitness = np.mean(fitnesses)
    print(f" Average fitness over {runs_per_config} runs: {avg_fitness:.2f}")

    tuning_results.append({
        "experiment": experiment_id,
        "L": L,
        "k": k,
        "c": c,
        "stop": stop,
        "fitnesses": fitnesses,
        "avg_fitness": avg_fitness
    })

    experiment_id += 1

# Find best config by average fitness
best_exp = max(tuning_results, key=lambda x: x["avg_fitness"])

print("\n=== Best Configuration ===")
print(f"Experiment {best_exp['experiment']}:")
print(f"L={best_exp['L']}, k={best_exp['k']}, c={best_exp['c']}, stop={best_exp['stop']}")
print(f"Average fitness: {best_exp['avg_fitness']:.2f}")

## Genetic Algorithms

In [8]:
import numpy as np
import random
import copy

from optimizers.genetic_algorithms.selection import selection
from optimizers.genetic_algorithms.mutation import mutation
from optimizers.genetic_algorithms.crossover import crossover
from optimizers.genetic_algorithms.genetic_algorithm import GeneticAlgorithm

crossover_options = ["single table swap", "table by table"]
selection_options = ["battle", "double_roullette"]
mutation_flags = [
    (True, False, False),
    (False, True, False),
    (False, False, True)
]

for crossover_type in crossover_options:
    for selection_type in selection_options:
        for swap, table_flip, relationship_augmenter in mutation_flags:

            print(f"\nTesting with crossover={crossover_type}, selection={selection_type}, "
                  f"mutations=(swap={swap}, table_flip={table_flip}, rel_aug={relationship_augmenter})")

            # Simple crossover wrapper inline
            def crossover_wrapper(ind1, ind2):
                return crossover(ind1, ind2, crossover=crossover_type)

            ga = GeneticAlgorithm(
                helper=helper,
                pop_size=30,
                num_gen=10,
                p_xo=0.2,
                p_m=0.6,
                selection_method=selection_type,
                elitism=True,
                n_elite=2,
                n_battles=3,
                p_pity=0.001,
                swap=swap,
                table_flip=table_flip,
                relationship_augmenter=relationship_augmenter,
                selection_func=selection,
                crossover_func=crossover_wrapper,
                mutation_func=mutation
            )

            best_fitness, best_solution = ga.run()

            print(f"Best fitness: {best_fitness}")
            print(f"First table in solution: {best_solution[0]}")


Testing with crossover=single table swap, selection=battle, mutations=(swap=True, table_flip=False, rel_aug=False)
gen 0
[[np.int64(12), np.int64(45), np.int64(35), np.int64(38), np.int64(23), np.int64(48), np.int64(14), np.int64(41)], [np.int64(24), np.int64(10), np.int64(44), np.int64(5), np.int64(39), np.int64(50), np.int64(49), np.int64(7)], [np.int64(6), np.int64(54), np.int64(62), np.int64(56), np.int64(33), np.int64(3), np.int64(22), np.int64(27)], [np.int64(34), np.int64(17), np.int64(16), np.int64(11), np.int64(64), np.int64(59), np.int64(8), np.int64(47)], [np.int64(63), np.int64(30), np.int64(40), np.int64(25), np.int64(31), np.int64(60), np.int64(55), np.int64(29)], [np.int64(13), np.int64(20), np.int64(61), np.int64(46), np.int64(53), np.int64(52), np.int64(2), np.int64(9)], [np.int64(51), np.int64(26), np.int64(18), np.int64(4), np.int64(1), np.int64(37), np.int64(42), np.int64(43)], [np.int64(57), np.int64(15), np.int64(28), np.int64(58), np.int64(36), np.int64(21), np.